In [1]:
#!unzip ./drive/MyDrive/single-person-pose-estimation/dataset/tfrecords.zip

In [2]:
import sys
import time
from datetime import datetime, timedelta
sys.path.append('/content/drive/MyDrive/single-person-pose-estimation')
from configs import default_config as cfg



---



---
Dataset


In [3]:
from dataset_builder import DatasetBuilder

In [4]:
ds_builder = DatasetBuilder(cfg)

Train dataset with 149813 examples.
Valid dataset with 6352 examples.




---



---
Model


In [5]:
from model.hourglass import HourGlassNet

In [6]:
HG = HourGlassNet(cfg.NUM_KEYPOINTS, cfg.HG_NUM_STACKS, cfg.HG_NUM_FILTERS, 
              (cfg.IMAGE_HEIGHT, cfg.IMAGE_WIDTH, 3), 
              (cfg.LABEL_HEIGHT, cfg.LABEL_WIDTH, cfg.NUM_KEYPOINTS))
model = HG.create_hg_model()

Created HourGlassmodel:
    1. 4 stacks.
    2. 12080324 parameters. Call object.get_summary() for more detail.
    




---



---
TRAINER




---
First train


In [7]:
from trainer import Trainer

In [8]:
trainer = Trainer(model, ds_builder, epochs = 3, learning_rate = 0.01, config = cfg)

In [9]:
trainer.train()

Start traing with:
    1. Current date 22-03-2022.
    2. Number of epochs 3.
    3. Learning rate 0.01.
    4. Batch size 16.
    

Learning rate for epoch 1 is 0.009999999776482582
Epoch 1/3
9363/9363 [==============================] - ETA: 0s - loss: 0.0037 - heatmap1_loss: 9.3616e-04 - heatmap2_loss: 9.1399e-04 - heatmap3_loss: 9.0634e-04 - heatmap4_loss: 9.0327e-04
Epoch 1: val_loss improved from inf to 0.00339, saving model to temp/checkpoints/22-03-2022-E3.cpkt
9363/9363 [==============================] - 2762s 293ms/step - loss: 0.0037 - heatmap1_loss: 9.3616e-04 - heatmap2_loss: 9.1399e-04 - heatmap3_loss: 9.0634e-04 - heatmap4_loss: 9.0327e-04 - val_loss: 0.0034 - val_heatmap1_loss: 8.7766e-04 - val_heatmap2_loss: 8.4468e-04 - val_heatmap3_loss: 8.3690e-04 - val_heatmap4_loss: 8.3025e-04

Learning rate for epoch 2 is 0.009999999776482582
Epoch 2/3
 585/9363 [>.............................] - ETA: 42:13 - loss: 0.0032 - heatmap1_loss: 8.2491e-04 - heatmap2_loss: 7.9254e-04 - h

KeyboardInterrupt: ignored

In [27]:
# from drive
!cp -r ./drive/MyDrive/single-person-pose-estimation/temp ./

In [28]:
# to drive
!cp -r ./temp ./drive/MyDrive/single-person-pose-estimation

In [23]:
!rm -r temp